[View in Colaboratory](https://colab.research.google.com/github/igoryamamoto/ws-tf/blob/master/Tensor_Flow_Course.ipynb)

# Digit Recognition with Tensor Flow

## Import dependencies

In [3]:
import numpy as np
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

## Convolutional Neural Network Definition

In [0]:
def cnn_model_fn(features, labels,mode):
  #input
  input_layer  = tf.reshape(features['x'], [-1,28,28,1])
  
  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding="same",
        activation=tf.nn.relu)
  
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
  
  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
    inputs=pool1,
    filters=64,
    kernel_size=[5, 5],
    padding="same",
    activation=tf.nn.relu)
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)
  
  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)
  dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=(mode == tf.estimator.ModeKeys.TRAIN))
  
  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)
  
  
  predictions = {
     # Generate predictions (for PREDICT and EVAL mode)
     "classes": tf.argmax(input=logits, axis=1),
     # Add `softmax_tensor` to the graph. It is used for PREDICT and by the `logging_hook`.
     "probabilities": tf.nn.softmax(logits, name="softmax_tensor"),
  }
  
  if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
  
  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
      optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
      train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
      return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

## Train and Evaluate MNIST example

In [0]:
# Load training and eval data
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

# Create the Estimator
mnist_classifier = tf.estimator.Estimator(model_fn=cnn_model_fn, model_dir="/tmp/mnist_convnet_model")

# Set up logging for predictions
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=50)

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])

# Evaluate the model and print results
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)
eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

Instructions for updating:
Please use tf.data.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting MNIST-data/t10k-labels-idx1-u

INFO:tensorflow:probabilities = [[0.10103126 0.09657289 0.10110503 0.09903058 0.09794539 0.10468317
  0.10957886 0.09882978 0.09421331 0.09700974]
 [0.10742753 0.11871902 0.09511614 0.09225898 0.09177843 0.11154287
  0.11636858 0.08526626 0.09852802 0.08299417]
 [0.10749424 0.10352323 0.09612851 0.08587757 0.09529715 0.10567147
  0.10710427 0.09909358 0.09855638 0.10125363]
 [0.09079933 0.11441863 0.09041522 0.09854399 0.08641054 0.0959039
  0.12797573 0.09307602 0.1023185  0.10013814]
 [0.09717491 0.10514337 0.09929725 0.0887337  0.08936427 0.11901335
  0.10590704 0.09913641 0.09651982 0.0997099 ]
 [0.0965658  0.10714559 0.09654834 0.09800942 0.09528873 0.09743547
  0.10723237 0.09305558 0.10565008 0.10306863]
 [0.09609998 0.11123443 0.09941878 0.09504549 0.09599253 0.09806543
  0.10447265 0.09977231 0.10184526 0.09805314]
 [0.09562736 0.10084406 0.10038322 0.09069914 0.09488664 0.10800751
  0.10691056 0.09464709 0.10405033 0.10394411]
 [0.09499989 0.10141717 0.10445099 0.09341747 0.0

INFO:tensorflow:probabilities = [[0.09709986 0.10193948 0.0984676  0.09226307 0.0804396  0.09895532
  0.11235978 0.09969655 0.1106374  0.10814127]
 [0.1086291  0.08816229 0.09753895 0.10094664 0.1105861  0.09173148
  0.10763711 0.09158944 0.09227533 0.11090362]
 [0.09546773 0.10247301 0.10166793 0.09430104 0.09747747 0.10475222
  0.11258581 0.0929979  0.09790911 0.10036783]
 [0.09864702 0.11173546 0.09347197 0.08617298 0.10330336 0.09829638
  0.11103921 0.09499823 0.10604174 0.09629362]
 [0.09453485 0.10462158 0.10391554 0.09210308 0.0881068  0.10303028
  0.11111595 0.09775592 0.10432176 0.10049419]
 [0.10460021 0.09724491 0.09835675 0.09825123 0.09482668 0.1022594
  0.10962629 0.08701008 0.10513132 0.10269311]
 [0.09605853 0.10094503 0.09711631 0.09357485 0.09263684 0.09595159
  0.1081685  0.10180606 0.1100214  0.10372088]
 [0.10120862 0.09643597 0.09533867 0.10151638 0.09200288 0.1049923
  0.10671878 0.10091539 0.10460693 0.09626407]
 [0.10180138 0.10170005 0.09358928 0.09326553 0.09

INFO:tensorflow:probabilities = [[0.10375492 0.09654199 0.10466365 0.09350328 0.09611151 0.09690715
  0.1034373  0.10196084 0.10198014 0.10113917]
 [0.10124101 0.09832637 0.0936693  0.08472046 0.0992647  0.10452666
  0.11266082 0.08409052 0.0930016  0.12849854]
 [0.1029182  0.10134001 0.08855978 0.10010483 0.09711438 0.09861287
  0.10462034 0.09532347 0.10740814 0.103998  ]
 [0.09957644 0.10282081 0.10176953 0.10226057 0.09207756 0.09726939
  0.10106921 0.09512486 0.10126299 0.1067687 ]
 [0.10987076 0.08942004 0.08616382 0.10189272 0.09286911 0.10845765
  0.10425254 0.09822311 0.10098994 0.10786032]
 [0.10030051 0.10142299 0.09747455 0.09311549 0.09828725 0.09824359
  0.10468989 0.09898035 0.10350178 0.10398368]
 [0.10343781 0.09594372 0.09510292 0.09987197 0.10525806 0.09910589
  0.09748664 0.09413037 0.10409447 0.10556812]
 [0.09639229 0.09821937 0.08830851 0.09680847 0.11429332 0.1000856
  0.10145434 0.098408   0.10327045 0.10275962]
 [0.09650617 0.09898718 0.10105625 0.10062474 0.0

INFO:tensorflow:probabilities = [[0.09988569 0.09766576 0.10069019 0.11048048 0.10435559 0.09894503
  0.10461934 0.08773974 0.10005797 0.09556016]
 [0.11007033 0.09307764 0.10189845 0.10129607 0.08465863 0.10501471
  0.09772157 0.10406214 0.10195991 0.10024048]
 [0.12223279 0.0866149  0.09026502 0.09563781 0.08935381 0.10254594
  0.10538092 0.08835234 0.10895841 0.11065808]
 [0.09269721 0.10485568 0.09557744 0.09519216 0.09633394 0.10361484
  0.10323812 0.10485416 0.10164069 0.10199577]
 [0.1015038  0.10170446 0.09233051 0.09594302 0.10170873 0.09501868
  0.10151404 0.09773845 0.10024608 0.11229215]
 [0.09560212 0.09617516 0.09662462 0.0878209  0.10146617 0.10395937
  0.10134289 0.0978469  0.09996288 0.11919905]
 [0.10856763 0.10669457 0.10055764 0.10548278 0.08951188 0.09348939
  0.09401917 0.09670176 0.10092649 0.10404874]
 [0.10402814 0.10392936 0.09484524 0.09258498 0.09009759 0.10943534
  0.10098659 0.09667508 0.11598798 0.09142978]
 [0.08567272 0.10898862 0.10322175 0.09707181 0.

...

INFO:tensorflow:probabilities = [[0.00123001 0.00490753 0.00060747 0.00063476 0.8711412  0.00903584
  0.00567033 0.01045229 0.01647621 0.07984441]
 [0.00005204 0.00084351 0.9755779  0.00408703 0.00010429 0.00009999
  0.00100712 0.01386174 0.00239243 0.00197397]
 [0.0000007  0.00000002 0.00000014 0.00007549 0.00000002 0.00000227
  0.         0.99991465 0.00000107 0.00000562]
 [0.00001303 0.000001   0.00205318 0.00000019 0.00005095 0.00000175
  0.997867   0.00000001 0.00001174 0.00000114]
 [0.00000213 0.00000023 0.00000678 0.00002106 0.9748003  0.00005796
  0.00038414 0.00010714 0.00180827 0.02281186]
 [0.00245066 0.00000405 0.00222299 0.01017263 0.00000388 0.00314082
  0.00001125 0.00001422 0.9818371  0.0001424 ]
 [0.00237773 0.00004157 0.00237979 0.00018708 0.00298304 0.02369564
  0.96344745 0.00010422 0.00470838 0.00007514]
 [0.00062746 0.00000617 0.00993404 0.00740442 0.10429687 0.00586059
  0.00054584 0.00251976 0.13617674 0.7326281 ]
 [0.00780852 0.00410283 0.00465494 0.12480015 0.

INFO:tensorflow:probabilities = [[0.00913065 0.11233388 0.00184895 0.49472502 0.00037566 0.03823872
  0.0010687  0.00384773 0.3330213  0.00540936]
 [0.00006202 0.00000098 0.00010663 0.00035656 0.0000382  0.00070298
  0.00008735 0.00001234 0.99670273 0.00193011]
 [0.00033029 0.97884446 0.00845475 0.00064162 0.00047223 0.00059333
  0.00146385 0.00117392 0.00787449 0.00015103]
 [0.00080933 0.97138804 0.00293907 0.00391369 0.00049261 0.00164841
  0.00158407 0.0086296  0.00829204 0.00030326]
 [0.00001558 0.00000922 0.00203103 0.01524879 0.00000445 0.00132505
  0.00000546 0.00000826 0.98100865 0.00034344]
 [0.00424888 0.00191302 0.00252248 0.00127208 0.00666201 0.01385635
  0.96710646 0.00007186 0.00144636 0.00090043]
 [0.02663504 0.7564159  0.0046367  0.00458187 0.00266513 0.0768364
  0.04861919 0.00567145 0.06918426 0.00475412]
 [0.00023602 0.00015364 0.00271179 0.00047451 0.00034445 0.00199023
  0.9934488  0.00000158 0.0006119  0.00002714]
 [0.0000978  0.00000206 0.0000095  0.07133842 0.0

INFO:tensorflow:probabilities = [[0.00003492 0.00001871 0.99895966 0.00071872 0.0000054  0.00003633
  0.00016798 0.00000977 0.00003949 0.00000907]
 [0.00006492 0.00000142 0.9981968  0.00045364 0.0004775  0.00005493
  0.00047928 0.00000022 0.00024447 0.00002679]
 [0.99997354 0.         0.00000226 0.00000398 0.         0.00001904
  0.00000002 0.00000045 0.00000004 0.00000069]
 [0.00087742 0.08973812 0.22392501 0.04643727 0.00076316 0.0001109
  0.00023961 0.60087854 0.00677605 0.03025394]
 [0.00001923 0.00000008 0.00056327 0.99915063 0.00000004 0.00002506
  0.00000006 0.00000006 0.00023284 0.00000874]
 [0.00000011 0.00000005 0.00000017 0.00000005 0.9996346  0.00007468
  0.00001226 0.00002591 0.00012144 0.00013069]
 [0.00007223 0.00001565 0.00001049 0.00095754 0.02162979 0.00015062
  0.00000445 0.02314213 0.00091041 0.95310664]
 [0.04360405 0.00000151 0.00103196 0.01063058 0.00034899 0.02991375
  0.00018099 0.0000692  0.9079283  0.00629079]
 [0.99872166 0.         0.00058238 0.00008603 0.0

INFO:tensorflow:probabilities = [[0.0004053  0.00000395 0.00107662 0.00058284 0.00050767 0.00250563
  0.00005183 0.00000169 0.99433535 0.00052912]
 [0.0008554  0.00217252 0.19066976 0.0009446  0.56794566 0.01778268
  0.1239569  0.07069375 0.0003081  0.02467056]
 [0.0049053  0.00025365 0.00496166 0.9045926  0.00000585 0.07468264
  0.00159578 0.00015091 0.00850719 0.00034449]
 [0.00313449 0.00792071 0.00091122 0.01999932 0.00705767 0.01663378
  0.00013569 0.42093873 0.00741792 0.5158504 ]
 [0.00010837 0.01499632 0.00309542 0.00014418 0.00038228 0.0011898
  0.97764903 0.00000147 0.0024097  0.00002337]
 [0.00034664 0.00035024 0.00130938 0.0027418  0.38973334 0.00086399
  0.00162873 0.00624765 0.00647329 0.5903049 ]
 [0.0000491  0.9828771  0.00157318 0.00483085 0.00071217 0.0002822
  0.00064408 0.00644208 0.001304   0.00128528]
 [0.71473676 0.00000156 0.00101319 0.00011045 0.00000782 0.28173125
  0.00198057 0.00010809 0.00025937 0.00005097]
 [0.00012559 0.0000058  0.00000459 0.0000835  0.00

INFO:tensorflow:probabilities = [[0.00012456 0.00000044 0.0000841  0.00007647 0.979727   0.00052814
  0.00074607 0.00017447 0.00096952 0.01756923]
 [0.01192841 0.00001196 0.00084733 0.00112868 0.01255442 0.8248774
  0.12779862 0.00000653 0.01923103 0.00161562]
 [0.0019154  0.00225006 0.26214167 0.00540451 0.38617495 0.03400883
  0.2614504  0.00208158 0.04022136 0.00435132]
 [0.001094   0.00000501 0.9940568  0.00034349 0.00000818 0.00014951
  0.00423321 0.00001149 0.00009689 0.00000139]
 [0.9993543  0.00000006 0.00003302 0.000024   0.00000086 0.00034434
  0.00002738 0.00000495 0.00019896 0.00001217]
 [0.9427324  0.00000013 0.0001651  0.0001806  0.00003111 0.01749106
  0.0002213  0.0197625  0.00415522 0.01526061]
 [0.99808747 0.0000001  0.00010958 0.00000435 0.00000467 0.0011046
  0.0002325  0.00019252 0.00001598 0.00024817]
 [0.00000717 0.00000039 0.00000066 0.00013676 0.04745478 0.00054446
  0.00000176 0.00523013 0.00532851 0.9412954 ]
 [0.00000002 0.00000817 0.00000074 0.00043844 0.98